In [2]:
from ipynb.fs.full.UtilitiesConceptDrift import *

In [2]:
test_set = OpenDataset("FullDataset.csv")
print(f"Number of tweets: {len(test_set)}")

Number of tweets: 33602


In [3]:
test_set

,timestamp,username,tweet,likes_count
0,2021-02-28 00:27:30,filipporiccio1,l hanno gia data questa disponibilita moderna...,4
1,2021-02-28 00:11:00,cavinadenise,in molti che non sanno che non c era ancora u...,0
2,2021-02-27 23:54:46,biber0n1,il vaccino deve essere un farmaco generico pr...,1
3,2021-02-27 23:42:48,icolomboa,due mesi fa di questi giorni tra tanto scettic...,0
4,2021-02-27 23:35:44,attilaazurerive,tanta voglia di iri e di ritorno ai vecchi dis...,0
...,...,...,...,...
35290,2021-05-01 05:13:10,25o319,allora qui sarebbe tempo di fare funzionare il...,6
35291,2021-05-01 04:05:42,yy01329,il boom dei vaccini raggiunta quota 500 mila a...,2
35292,2021-05-01 03:49:34,claudiomarang1,del vaccino cubano ben che vada vedremo la lu...,1
35293,2021-05-01 03:08:15,bludichina,e il target primario dei vaccini attenuare i ...,0


In [4]:
test_set = test_set.sort_values(by=['likes_count'], ascending=False)

In [5]:
data = search_tweet_Period(test_set, "2021-04-01", "2021-04-30").copy()

In [6]:
data.to_csv("ConceptDriftAprile.csv")

In [ ]:
#Do classification by hand

In [23]:
#Reload data
data = pd.read_csv("ConceptDriftFebbraio.csv",sep=',',usecols=['timestamp','username','tweet','label'])
data=data.dropna()
data2 = pd.read_csv("ConceptDriftMarzo.csv",sep=',',usecols=['timestamp','username','tweet','label'])
data2=data.dropna()
training_set = pd.read_csv("training_set.csv",sep=',',usecols=['timestamp','username','tweet','label'])

In [24]:
#Merge the all the data
new_trainin_set = pd.concat([training_set,data,data2])
new_trainin_set.label=new_trainin_set.label.astype(int)

In [25]:
#print(new_trainin_set)
print(f"Number of tweets: {len(new_trainin_set)}")

Number of tweets: 1026


In [26]:
#Training the model
clf = Pipeline([
    ('vect', StemmedCountVectorizer(min_df=3, analyzer="word", stop_words = set(stopwords.words('italian')), ngram_range = (1,1))),
    ('tfidf', TfidfTransformer(smooth_idf=True,use_idf=True)),
    ('clf',BernoulliNB()),
])
clf.fit(new_trainin_set.tweet, new_trainin_set.label)

Pipeline(steps=[('vect',
                 StemmedCountVectorizer(min_df=3,
                                        stop_words={'a', 'abbia', 'abbiamo',
                                                    'abbiano', 'abbiate', 'ad',
                                                    'agl', 'agli', 'ai', 'al',
                                                    'all', 'alla', 'alle',
                                                    'allo', 'anche', 'avemmo',
                                                    'avendo', 'avesse',
                                                    'avessero', 'avessi',
                                                    'avessimo', 'aveste',
                                                    'avesti', 'avete', 'aveva',
                                                    'avevamo', 'avevano',
                                                    'avevate', 'avevi', 'avevo', ...})),
                ('tfidf', TfidfTransformer()), ('clf', BernoulliNB())])

In [27]:
#Save the new model 
#CAMBIARE IL NOME!!!!!!!!!!!!!
joblib.dump(clf, 'modelMarzo.pkl', compress=9)

['modelMarzo.pkl']

In [3]:
#Load next month tweet classified
test_set = pd.read_csv("ConceptDriftMarzo.csv",sep=',',usecols=['timestamp','username','tweet','label'])
test_set = test_set.dropna()
test_set.label=test_set.label.astype(int)

In [58]:
#print(test_set)
print(f"Number of tweets: {len(test_set)}")

Number of tweets: 60


In [59]:
#Do classification with new model
test_set["predicted"]= clf.predict(test_set.tweet)

In [60]:
print(test_set)

               timestamp         username  \
0    2021-03-18 17:40:12   robertoburioni   
1    2021-03-18 19:25:57  gassmangassmann   
3    2021-03-16 15:58:33      matteorenzi   
4    2021-03-18 17:15:53   robertoburioni   
5    2021-03-11 14:31:54   robertoburioni   
6    2021-03-13 13:03:03   robertoburioni   
7    2021-03-04 14:23:59   francescocosta   
8    2021-03-13 18:55:24  alanelettronico   
9    2021-03-16 16:53:00       catlatorre   
10   2021-03-15 17:44:24     simonespetia   
11   2021-03-12 13:13:12      mantinirita   
12   2021-03-17 13:37:35   robertoburioni   
13   2021-03-16 07:54:29  antonellaviol17   
14   2021-03-11 20:42:39    claudiocerasa   
15   2021-03-16 14:03:22  stanzaselvaggia   
16   2021-03-29 16:14:47   robertoburioni   
17   2021-03-10 09:17:09         fdragoni   
18   2021-03-17 12:54:17    claudiocerasa   
19   2021-03-18 17:08:56   robertoburioni   
20   2021-03-14 19:42:44   aifa_ufficiale   
22   2021-03-01 23:32:47    lucianocapone   
23   2021-

In [31]:
#Benchmark new model
benchmark(test_set["label"], test_set["predicted"])

accuracy : 0.4666666666666667
f_score : 0.4460563077584354

[[14  3  3]
 [ 4 10  6]
 [ 9  7  4]]
Precision,Recall,f_score
 Classe 0:  [0.51851852 0.5        0.30769231] 
 Classe 1:  [0.7 0.5 0.2] 
 Classe 2:  [0.59574468 0.5        0.24242424] 
 Support:   [20 20 20]


In [4]:
#Load old model
clf2 = joblib.load('InitialModel.pkl')

In [6]:
#Do classification with old model
test_set["predicted_old"]= clf2.predict(test_set.tweet)

In [7]:
print(test_set["predicted_old"])

0      2
1      1
3      2
4      1
5      2
6      2
7      2
8      2
9      2
10     2
11     2
12     2
13     2
14     2
15     2
16     2
17     2
18     2
19     2
20     2
22     2
23     2
24     2
25     2
26     2
28     2
29     2
30     2
31     2
32     2
33     2
34     2
35     2
36     2
37     2
38     2
39     2
40     0
42     2
43     2
46     2
48     2
50     2
53     2
54     2
57     2
59     2
64     2
77     2
86     2
91     2
93     2
94     2
107    2
119    2
122    2
137    2
153    2
159    2
180    2
Name: predicted_old, dtype: object


In [8]:
test_set.predicted_old=test_set.label.astype(int)
print(test_set)

               timestamp         username  \
0    2021-03-18 17:40:12   robertoburioni   
1    2021-03-18 19:25:57  gassmangassmann   
3    2021-03-16 15:58:33      matteorenzi   
4    2021-03-18 17:15:53   robertoburioni   
5    2021-03-11 14:31:54   robertoburioni   
6    2021-03-13 13:03:03   robertoburioni   
7    2021-03-04 14:23:59   francescocosta   
8    2021-03-13 18:55:24  alanelettronico   
9    2021-03-16 16:53:00       catlatorre   
10   2021-03-15 17:44:24     simonespetia   
11   2021-03-12 13:13:12      mantinirita   
12   2021-03-17 13:37:35   robertoburioni   
13   2021-03-16 07:54:29  antonellaviol17   
14   2021-03-11 20:42:39    claudiocerasa   
15   2021-03-16 14:03:22  stanzaselvaggia   
16   2021-03-29 16:14:47   robertoburioni   
17   2021-03-10 09:17:09         fdragoni   
18   2021-03-17 12:54:17    claudiocerasa   
19   2021-03-18 17:08:56   robertoburioni   
20   2021-03-14 19:42:44   aifa_ufficiale   
22   2021-03-01 23:32:47    lucianocapone   
23   2021-

In [10]:
benchmark(test_set["label"].astype(int), test_set["predicted_old"].astype(int))

accuracy : 0.35
f_score : 0.22510822510822512

[[ 0  0 20]
 [ 0  2 18]
 [ 1  0 19]]
Precision,Recall,f_score
 Classe 0:  [0.         1.         0.33333333] 
 Classe 1:  [0.   0.1  0.95] 
 Classe 2:  [0.         0.18181818 0.49350649] 
 Support:   [20 20 20]
